In [15]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from decimal import Decimal

#Graphing variables
page_width = (8.5, 5)

In [16]:
#Calculate accurate floating point uncertainties using the Decimal module

#Uncertainty percentages recorded from 
#ftp://ftp.pasco.com/Support/Documents/english/CI/CI-6630/012-08541b.pdf
a_sensor = 0.0001

''' Finds the first significant digit in Decimal object n.
'''
def firstdigit(n):
    abs_n = abs(n)
    place = 0
    if (abs_n >= Decimal('1.0')):
        while (abs_n >= Decimal('10.0')):
            abs_n = Decimal.shift(abs_n, -1)
            place -= 1
    else:
        while (abs_n < Decimal('1.0')):
            abs_n = Decimal.shift(abs_n, 1)
            place += 1
    return round(n, place)

''' Finds the last significant digit in Decimal object n.
'''
def lastdigit(n):
    place = 0
    while (n % Decimal('1.0') == Decimal('0.0')):
        n = Decimal.shift(n, -1)
    while (n % Decimal('1.0') != Decimal('0.0')):
        n = Decimal.shift(n, 1)
        place -= 1
    return place

''' Calculates the maximum uncertainty by taking the larger between the error of
    accuracy and the error of precision.
    Error of accuracy is rounded to one significant digit.
'''
def finderror(x, a):
    dec_x = Decimal(str(np.abs(x)))
    dec_a = Decimal(str(a))
    err_a = firstdigit(dec_x * dec_a)
    err_p = Decimal('1.0') * Decimal(str(10.0**(lastdigit(dec_x))))
    return (float)(max(err_a, err_p))

In [44]:
#Function definitions

''' Plot a function, with optional parameters.
'''
def plot_function(data, weight, filename, title, directory='graphs/', figsize=page_width,
                  plot_error=False, second_data=None, second_label=None):
    plt.figure(figsize=figsize)
    if plot_error:
        plt.errorbar(data[:,0], data[:,1], yerr=weight, elinewidth=0.8, capthick=1.2, capsize=2.0, 
                     fmt='none', label='data')
    else:
        plt.plot(data[:,0], data[:,1], label='data')
    if second_data: 
        plt.plot(second_data[0], second_data[1], label=second_label)
        plt.legend()
    plt.xlabel('Sensor Position (degrees)')
    plt.ylabel('Light Intensity (Volts)')
    plt.title(title)
    plt.savefig(directory+filename+'.png', bbox_inches='tight')

''' Finds the peaks of the data, assuming they are the two greatest values. Returns the angles at which they occur.
'''
def find_peaks(data):
    max1, max2 = 0, 0
    for point in data:
        if point[0] > 10 and point[0] < 30 and point[1] > max1: 
            max1 = point[1]
            angle1 = point[0]
    for point in data:
        if point[0] > 60 and point[0] < 75 and point[1] > max2: 
            max2 = point[1]
            angle2 = point[0]
    return (angle1, angle2)

''' Model function for blackbody radiation, based on Wien's Law. 
'''

" Model function for blackbody radiation, based on Wien's Law. \n"

In [23]:
#Load the data
directory = 'data/'
filenames = ['4.0V0.384A', '4.5V0.412A', '5.0V0.437A', '5.5V0.463A', '6.0V0.481A', '6.5V0.508A', 
             '7.0V0.531A', '7.5V0.549A', '8.0V0.57A', '8.5V0.585A', '9.0V0.606A', '9.5V0.624A', 
             '10.0V0.639A10x', '10.0V0.642A1x', '10.0V0.645A']

data, u = [], []
for i in range(len(filenames)):
    data.append(np.loadtxt(directory+filenames[i]+'.txt', skiprows=2))
data = np.array(data)

#Clear zero data at beginning of file
for i in range(len(filenames)):
    while (Decimal((float)(data[i][0][0])) == Decimal('0.0')): data[i] = np.delete(data[i], (0), axis=0)
    u.append([finderror(data[i][j,1], a_sensor) for j in range(len(data[i]))])

data, u = np.array(data), np.array(u)

In [ ]:
%%time
#CPU times: user 8.78 s, sys: 1.64 s, total: 10.4 s Wall time: 5.54 s

#Plot data
for i in range(len(data)):
    plot_function(data[i], u[i], filenames[i], 'Light Intensity vs Sensor Position, '+filenames[i],
                  directory='graphs/raw/')
    plot_function(data[i], u[i], filenames[i]+'eb', 'Light Intensity vs Sensor Position, '+filenames[i], 
                  plot_error=True, directory='graphs/rawerror/')

In [48]:
angles = []

for entry in data:
    angles.append(find_peaks(entry))

angles = np.array(angles)

for pair in angles:
    print(pairs[1]-pars[0])

NameError: name 'pairs' is not defined